In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI TensorBoard: Delete Outdated TensorBoard Experiments

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/delete_outdated_tensorboard_experiments.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/delete_outdated_tensorboard_experiments.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/vertex-ai-samples/main/notebooks/official/experiments/delete_outdated_tensorboard_experiments.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview

Vertex AI will have a new TensorBoard billing model. From August 2023, it will change from monthly `$300/user` to monthly `$10/GB`. In preparation for this change, users need to delete old TensorBoard Experiments to avoid unnecessary storage costs when the pricing change takes place.

Learn more about [Vertex AI Tensorboard](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview).

### Objective

In this tutorial, you learn how to delete outdated TensorBoard Experiments to avoid unnecessary storage costs.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Tensorboard

The steps performed include:

- How to delete the TB Experiment with a predefined key-value label pair `<label_key, label_value>`

- How to delete the TB Experiments created before the  `create_time`

- How to delete the TB Experiments created before the  `update_time`

### Dataset

No dataset has been used.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), and [Cloud Storage pricing](https://cloud.google.com/storage/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook.

In [ ]:
# Install the packages
import os

if not os.getenv("IS_TESTING"):
    USER = "--user"
else:
    USER = ""
! pip3 install {USER} --upgrade google-cloud-aiplatform

### Colab only: Uncomment the following cell to restart the kernel.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant permissions to your service account at https://cloud.google.com/marketplace/docs/grant-service-account-access.

### Import libraries

In [ ]:
from google.cloud import aiplatform

### Define constants

Define variables you use in this tutorial. In particular, you set

- `CREATE_TIME_CUT` ie delete tensorboard experiments that were created before CREATE_TIME_CUT. For example, `2022-12-31`.

- `UPDATE_TIME_CUT` ie delete tensorboard experiments that were created before UPDATE_TIME_CUT. For example, `2022-12-31`.

- `DETAILED_LOG` ie a booled variable to see Tensorboard deletion progress. If True, it shows progress by experiments. Otherwise, it reports progress per 100 experiments.

In [ ]:
CREATE_TIME_CUT = ""  # @param {type:"string"}

UPDATE_TIME_CUT = ""  # @param {type:"string"}

DETAILED_LOG = True  # @param {type: "boolean"}

### Define Helpers

In [ ]:
def clean_up_by_label(tensorboard_instance, label_key, label_value):
    """Delete the TB Experiment with the key-value label pair <label_key, label_value>"""
    # List tensorboard experiments
    tensorboard_experiments = aiplatform.TensorboardExperiment.list(
        tensorboard_name=tensorboard_instance.resource_name
    )
    # Get the number of tensorboard experiments
    num_tensorboard_experiments = len(tensorboard_experiments)
    # For each experiment
    for i in range(num_tensorboard_experiments):
        tensorboard_experiment = tensorboard_experiments[i]
        if detailed_log or (i % 100 == 0):
            print(
                f">>>checking TB experiment [{i + 1}/{num_tensorboard_experiments}]: {tensorboard_experiment.resource_name}"
            )
        # Get experiment labels
        labels = tensorboard_experiment.labels
        # Filter by label
        if label_key in labels and labels[label_key] == label_value:
            # Delete experiment
            tensorboard_experiment.delete()


def clean_up_by_create_time(tensorboard_instance, create_time_cut):
    """Delete the TB Experiment with the `create_time`"""
    # List tensorboard experiments
    tensorboard_experiments = aiplatform.TensorboardExperiment.list(
        tensorboard_name=tensorboard_instance.resource_name, order_by="create_time"
    )
    # Get the number of tensorboard experiments
    num_tensorboard_experiments = len(tensorboard_experiments)
    # For each experiment
    for i in range(num_tensorboard_experiments):
        tensorboard_experiment = tensorboard_experiments[i]
        if detailed_log or (i % 100 == 0):
            print(
                f">>> checking TB experiment [{i + 1}/{num_tensorboard_experiments}]: {tensorboard_experiment.resource_name}"
            )
        # Filter by create_time
        if str(tensorboard_experiment.create_time) < create_time_cut:
            # Delete experiment
            tensorboard_experiment.delete()
        else:
            break


def clean_up_by_update_time(tensorboard_instance, update_time_cut):
    """Delete the TB Experiment with with the `update_time`"""
    # List tensorboard experiments
    tensorboard_experiments = aiplatform.TensorboardExperiment.list(
        tensorboard_name=tensorboard_instance.resource_name, order_by="update_time"
    )
    # Get the number of tensorboard experiments
    num_tensorboard_experiments = len(tensorboard_experiments)
    # For each experiment
    for i in range(num_tensorboard_experiments):
        tensorboard_experiment = tensorboard_experiments[i]
        if detailed_log or (i % 100 == 0):
            print(
                f">>> checking TB experiment [{i + 1}/{num_tensorboard_experiments}]: {tensorboard_experiment.resource_name}"
            )
        # Filter by update_time
        if str(tensorboard_experiment.update_time) < update_time_cut:
            tensorboard_experiment.delete()
        else:
            break

## Delete outdated Tensorboard Experiments

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

### Delete an Tensorboard instance

To delete a Tensorboard instance, you need `TENSORBOARD_INSTANCE` ID which uniquely identifies the Tensorboard instance where you run experiments.

To get the Tensorboard instance ID, you can either

- go to the cloud console UI, Vertex AI > Experiments > Tensorboard Instances, or
- use the list command below to list all TensorBoard instances for your project and region.


In [ ]:
tensorboard_instances = aiplatform.Tensorboard.list(project=PROJECT_ID, location=REGION)
print(tensorboard_instances)

Set the tensorboard instance id for which you want to delete experiments.

In [ ]:
TENSORBOARD_INSTANCE_ID = ""  # @param {type:"string"}

TENSORBOARD_INSTANCE = aiplatform.Tensorboard(
    project=PROJECT_ID, location=REGION, tensorboard_name=TENSORBOARD_INSTANCE_ID
)
print(TENSORBOARD_INSTANCE)

### Delete a Tensorboard Experiment with the key-value label pair

You delete a Tensorboard experiment using a predefined `label_key` and `label_value`. For example, you may have assigned `delete` label key and `true` label value to indicate all Tensorboard experiments you want to delete.


In [ ]:
LABEL_KEY = "delete"  # @param {type:"string"}
LABEL_VALUE = "true"  # @param {type:"string"}

In [ ]:
clean_up_by_label(TENSORBOARD_INSTANCE, LABEL_KEY, LABEL_VALUE)

### Delete a Tensorboard Experiment with `create_time`

You delete a Tensorboard experiment using `create_time` field

In [ ]:
clean_up_by_create_time(TENSORBOARD_INSTANCE, CREATE_TIME_CUT)

### Delete a Tensorboard Experiment with `update_time`

You delete a Tensorboard experiment using a predefined `update_time` field

In [ ]:
clean_up_by_update_time(TENSORBOARD_INSTANCE, UPDATE_TIME_CUT)